[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/floleuerer/ml-tutorials/blob/main/01_fastai_pets.ipynb)

# Vorbereitungen

Python-Pakete von [fast.ai](https://www.fast.ai) installieren und importieren.

In [ ]:
!pip install -Uq fastai

In [ ]:
from fastai.vision.all import *

**Wichtig!** Wenn die folgende Zelle "False" ausgibt, ist keine GPU vorhanden -> Bitte prüfe die o. g. Schritte noch einmal und starte ggf. das Notebook neu. Bei "True" hast du eine GPU-Instanz und kannst weiter machen.

In [ ]:
torch.cuda.is_available()

# Class Activation Map (CAM)

Die Class Acitvation Map (CAM) wurde von Bolei Zhou in ["Learning Deep Features for Discriminative Localization"](https://arxiv.org/abs/1512.04150). Es verwendet die Ausgabe des letzten Convolution Layers (kurz vor dem Pooling-Layer) zusammen mit der Prediction / Output, um eine Heatmap-Visualisierung zu erhalten, warum das Modell seine Entscheidung getroffen hat. Dies ist ein nützliches Werkzeug zur Interpretation.
Genauer gesagt haben wir an jeder Position unseres letzten Convolution Layers so viele Filter wie im letzten Linear-Layer. Wir können daher das Skalarprodukt dieser Aktivierungen mit den letzen Parametern berechnen, um für jeden Ort auf unserer Feature-Map die Bewertung des Features zu erhalten, mit dem eine Entscheidung getroffen wurde.

# Daten laden

In [ ]:
path = untar_data(URLs.PETS)

In [ ]:
pets = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                 get_items=get_image_files, 
                 splitter=RandomSplitter(),
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224))
dls = pets.dataloaders(untar_data(URLs.PETS)/"images")

In [ ]:
dls.show_batch()

# Modell trainieren

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(1)

# Ergebnisse anzeigen

Mit `learn.predict()` können wir ein einzelnes Bild von unserem Modell klassifizieren lassen.

In [ ]:
img_id = 121
img = dls.valid_ds[img_id][0]
lbl = dls.valid_ds[img_id][1]
learn.predict(img)

Liegt das Modell richtig? Die folgende Zelle zeigt das Bild an:

In [ ]:
img, dls.vocab[lbl]

# Was führt zu dieser Klassifizierung?

In [ ]:
x, = first(dls.test_dl([img]))
img

In [ ]:
lbl, dls.vocab

In [ ]:
with hook_output(learn.model[0]) as h:
    with torch.no_grad(): output = learn.model.eval()(x)

In [ ]:
act = h.stored[0]

In [ ]:
cam_map = torch.einsum('ck,kij->cij', learn.model[1][-1].weight, act)
cam_map.shape

For each image in our batch, and for each class, we get a 7×7 feature map that tells us where the activations were higher and where they were lower. This will let us see which areas of the pictures influenced the model's decision.
For instance, we can find out which areas made the model decide this animal was a cat (note that we need to decode the input x since it's been normalized by the DataLoader, and we need to cast to TensorImage since at the time this book is written PyTorch does not maintain types when indexing—this may be fixed by the time you are reading this):

In [ ]:
x_dec = TensorImage(dls.train.decode((x,))[0][0])
_,ax = plt.subplots()
x_dec.show(ctx=ax)
ax.imshow(cam_map[1].detach().cpu(), alpha=0.6, extent=(0,224,224,0),
              interpolation='bilinear', cmap='magma');

The areas in bright yellow correspond to high activations and the areas in purple to low activations.